Voreinstellung

In [1]:
#r "C:\Aktuell_BoSSS_2\experimental\internal\src\private-ak\L4-application\StokesHelical_Ak\bin\Release\net6.0\BoSSSpad.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
Init();


In [2]:
#r "C:\Aktuell_BoSSS_2\experimental\internal\src\private-ak\L4-application\StokesHelical_Ak\bin\Release\net6.0\StokesHelical_Ak.dll"

In [3]:
using StokesHelical_Ak;

Voreinstellungen

In [4]:
ExecutionQueues

index,type,value
0,BoSSS.Application.BoSSSpad.SlurmClient,"SlurmClient Lb2 : sa90saxi@lcluster16.hrz.tu-darmstadt.de, Slurm account: special00006"
1,BoSSS.Application.BoSSSpad.MsHPC2012Client,"MS HPC client Default @DC3, @\\dc3\userspace\akbari\cluster\binaries"
2,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,MiniBatchProcessor client @\\dc3\userspace\akbari\Local\binaries


In [5]:
var myBatch = ExecutionQueues[1];
myBatch

RuntimeLocation,win\amd64
AdditionalEnvironmentVars,[ ]
DeploymentBaseDirectory,\\dc3\userspace\akbari\cluster\binaries
DeployRuntime,True
Name,Default
DotnetRuntime,dotnet
Username,FDY\akbari
NumOfAdditionalServiceCores,0
NumOfAdditionalServiceCoresMPISerial,0
NumOfServiceCoresPerMPIprocess,0
ServerName,DC3


## Projekt Name

In [6]:
string proj = "H_P";

In [7]:
BoSSSshell.WorkflowMgm.Init(proj,myBatch);

Project name is set to 'H_P'.
Opening existing database '\\dc3\userspace\akbari\cluster\H_P'.


In [8]:
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();

In [9]:
var myDb = BoSSSshell.WorkflowMgm.DefaultDatabase;

## Loading job

In [10]:
myDb = BoSSSshell.WorkflowMgm.DefaultDatabase;

In [11]:
myDb

{ Session Count = 1; Grid Count = 2; Path = \\dc3\userspace\akbari\cluster\H_P }

In [12]:
BoSSSshell.WorkflowMgm.Sessions

#0: H_P	BDF_1_LSol_PARDISO_Grid_regular_rMin_0_Mult_one_PRP_True_64_x_64_DGd_4_#MPI_4_Amp_5_dt_0.0001227184630308513*	5/10/2024 1:15:04 PM	62491359...


In [33]:
var fisch = BoSSSshell.WorkflowMgm.Sessions[0];

In [ ]:
fisch.Export(2)

In [35]:
var outPath = fisch.Export().WithSupersampling(1).Do();

Starting export process... Data will be written to the directory: C:\Users\akbari\AppData\Local\BoSSS\plots\sessions\H_P__TSFP_H_P_BDF3_Gridregular_degree=4_noOfCellsR=32_noOfCellsXi=32_Mar19_1704__fbcc2e9e-e40c-41f2-9403-88e172876b37
